In [8]:
from sklearn.model_selection import ParameterGrid
import time
import numpy as np
import keras

In [2]:
X_train = np.load('X_train.npy')  # Load your preprocessed training data
y_train = np.load('y_train.npy')
X_val = np.load('X_val.npy')  # Load your validation data
y_val = np.load('y_val.npy')
X_test = np.load('X_test.npy')  # Load your test data
y_test = np.load('y_test.npy')

In [3]:
# Define the parameter grid
param_grid = {
    "filters": [32, 64],
    "kernel_size": [3, 5],
    "dropout_rate": [0.2, 0.4],
    "batch_size": [16, 32],
    "learning_rate": [0.001, 0.0005]
}

In [4]:
# Create a list of all parameter combinations
param_combinations = list(ParameterGrid(param_grid))
print(f"Total combinations to try: {len(param_combinations)}")

Total combinations to try: 32


In [5]:
# Limit the number of combinations to try (adjust based on your PC specs)
max_combinations = 5
param_combinations = param_combinations[:max_combinations]

# Results storage
results = []

In [9]:
def make_model(input_shape, filters, kernel_size, dropout_rate):
    input_layer = keras.layers.Input(input_shape)
    x = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding="same")(input_layer)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.Dropout(dropout_rate)(x)

    x = keras.layers.GlobalAveragePooling1D()(x)
    output_layer = keras.layers.Dense(num_classes, activation="softmax")(x)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    return model

In [11]:
num_classes = len(np.unique(y_train))
for params in param_combinations:
    print(f"\nTraining with parameters: {params}")
    start_time = time.time()

    # Create model
    model = make_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        filters=params["filters"],
        kernel_size=params["kernel_size"],
        dropout_rate=params["dropout_rate"]
    )

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=params["learning_rate"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"]
    )

    # Train the model (fewer epochs for grid search)
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,  # Use fewer epochs for speed
        batch_size=params["batch_size"],
        verbose=1
    )

    # Evaluate on the validation set
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    elapsed_time = time.time() - start_time

    # Store the results
    results.append({
        "params": params,
        "val_loss": val_loss,
        "val_acc": val_acc,
        "time": elapsed_time
    })

    print(f"Validation Accuracy: {val_acc:.4f} | Time: {elapsed_time:.2f}s")


Training with parameters: {'batch_size': 16, 'dropout_rate': 0.2, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.001}
Epoch 1/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.6698 - sparse_categorical_accuracy: 0.6137 - val_loss: 0.4697 - val_sparse_categorical_accuracy: 0.7570
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.4697 - sparse_categorical_accuracy: 0.8080 - val_loss: 0.3954 - val_sparse_categorical_accuracy: 0.8530
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.4217 - sparse_categorical_accuracy: 0.8378 - val_loss: 0.3840 - val_sparse_categorical_accuracy: 0.8549
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.4166 - sparse_categorical_accuracy: 0.8394 - val_loss: 0.3715 - val_sparse_categorical_accuracy: 0.8722
Epoch 5/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.3867 - sparse_categorical_accuracy: 0.8548 - val_loss: 0.3583 - val_sparse_categorical_accuracy: 0.8665
Epoch 6/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 2

In [12]:
best_result = max(results, key=lambda x: x["val_acc"])
print("\nBest Parameters and Results:")
print(best_result)


Best Parameters and Results:
{'params': {'batch_size': 16, 'dropout_rate': 0.2, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.001}, 'val_loss': 0.3354590833187103, 'val_acc': 0.8856868147850037, 'time': 24.244338274002075}


In [13]:
from sklearn.metrics import classification_report
import keras

# Use the best parameters
best_params = {
    "batch_size": 16,
    "dropout_rate": 0.2,
    "filters": 32,
    "kernel_size": 3,
    "learning_rate": 0.001
}

# Define a new model with the best parameters
def make_model_with_best_params(input_shape, filters, kernel_size, dropout_rate, num_classes):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    gap = keras.layers.Dropout(dropout_rate)(gap)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

# Build and compile the model
model = make_model_with_best_params(
    input_shape=(X_train.shape[1], X_train.shape[2]),
    filters=best_params["filters"],
    kernel_size=best_params["kernel_size"],
    dropout_rate=best_params["dropout_rate"],
    num_classes=num_classes
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=best_params["learning_rate"]),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"]
)

# Train the model using the best batch size
model.fit(
    X_train, y_train,
    batch_size=best_params["batch_size"],
    epochs=50,  # A small number of epochs since it's already tuned
    validation_data=(X_val, y_val),
    verbose=1
)

# Evaluate the model on the test set
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Generate the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Epoch 1/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - loss: 0.5642 - sparse_categorical_accuracy: 0.7073 - val_loss: 0.3771 - val_sparse_categorical_accuracy: 0.8876
Epoch 2/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.3526 - sparse_categorical_accuracy: 0.8570 - val_loss: 0.2829 - val_sparse_categorical_accuracy: 0.9174
Epoch 3/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 0.2971 - sparse_categorical_accuracy: 0.8981 - val_loss: 0.2708 - val_sparse_categorical_accuracy: 0.9212
Epoch 4/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.3164 - sparse_categorical_accuracy: 0.8972 - val_loss: 0.3444 - val_sparse_categorical_accuracy: 0.9116
Epoch 5/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.2795 - sparse_categorical_accuracy: 0.9153 - val_loss: 0.2686 - val_sparse_categorical_accuracy: 0.9376
Epoch 6/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.2774 - sparse_categorical_accuracy: 0.9152 - val_loss: 0.2268 - val_sparse_categorical_accuracy: 0.938

In [16]:
# Save the trained model
model.save("best_model.keras")